# MyFitnessPal Diary Parser

In [1]:
# Libraries
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import datetime
import pandas as pd
import pickle
import logging

from configparser import ConfigParser
import os

# Notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)

# Logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO) # DEBUG

In [2]:
%%capture
# Parse parameters in config.ini
config = ConfigParser()
configfilePath = (os.path.join(os.getcwd(), 'config.ini'))
config.read(configfilePath)

input_dir = config.get('GLOBAL', 'input_folder')
input_file = config.get('GLOBAL', 'input_file')
data_file = input_dir + input_file
output_dir = config.get('GLOBAL', 'output_folder')
output_file = config.get('GLOBAL', 'output_file')

# Functions

In [3]:
def get_report_meta(html):
    """
    Purpose: Grab metadata from a MyFitnessPal diary.
    
    The result dataframe includes one row created with:
    * Date Range
    * Food statistic names (e.g. [Calories, Carbs, Protein, ...])
    * Exercise statistic names (e.g. [Calories, Minutes, Sets, ...])
      
    Args:
        html (BeautifulSoup): MyFitnessPal diary 
        
    Returns:
        dataframe
    """
    logging.debug('get_report_meta ...')
    result_dict = {}
    
    # Grab date range
    # "<input ... id="from" ... value="2017-11-01">
    result_dict['Date From'] = datetime.strptime(\
                [x["value"] for x in html.findAll("input", {"id": "from"})][0], \
                '%Y-%m-%d').date()
    result_dict['Date To'] = datetime.strptime(\
                [x["value"] for x in html.findAll("input", {"id": "to"})][0], \
                '%Y-%m-%d').date()
  
    
    # Extract dataset types (e.g. 'Foods' and 'Exercises').
    # Types are declared in the first <td> column of each <thead> section.
    # "<thead><tr><td class="first">Foods</td><td>Calories</td><td>...</td></tr></thead>"
    table_type_list = []   
    table_dict_list = []
    
    for thead in html.find_all('thead'):
        
        # Get column headers from this <thead>
        columns = thead.find('tr').findChildren('td')

        # First column = Data set type ('Foods' or 'Exercises')
        table_type = columns[0].get_text()  

        # Record unique data set types and their associated statistic types 
        if not table_type in table_type_list:

            table_type_list.append(table_type)

            # Remaining table columns headers statistic names (e.g. 'Carbs', 'Minutes).
            # List them.
            this_table_stats = {}
            this_table_stats['Stats List'] = [x.get_text() for x in columns[1:]]
            result_dict[table_type] = this_table_stats
              
    return pd.DataFrame(result_dict)
    

In [4]:
def get_daily_logs(html, totals_only=True):
    """
    Purpose: Parse HTML into embedded dictionaries. 
    
    Notes regarding the HTML parse:
    
    Log dates are tagged with <h2>, followed by that day's log entries: 
    
    * 'Foods' and 'Exercises' log entries are each tagged by <table>. The tables
    contain individual food items and exercises.
    
    * 'Food Notes' and 'Exercise Notes' are tagged with <h4>.
   
    Args:
        html (BeautifulSoup): MyFitnessPal diary 
        totals_only (boolean): return only daily statistic totals if True, 
                               otherwise include log entry details
        
    Returns:
        dictionary (embedded)
    """    

    logging.debug('get_daily_logs ...')
    result_dict = {x:[] for x in ['Foods', 'Exercises', 'Notes']}


    # Get 1st <h2> (log date), then find all its siblings.
    # Siblings of interest are <table> (log entires), <h4> (notes), 
    # and <h2> (the next log date).
    h2 = html.find('h2', {'id':'date'})
    h2_date = h2.get_text()  

    for tag in html.find('h2').next_siblings:

        # If <table> then parse table into this day's dictionary.
        if tag.name == 'table':
            this_table_dict, this_table_type = table_to_dict(tag, totals_only)
            this_table_dict['Date'] = h2_date
            
            result_dict[this_table_type].append(this_table_dict)   
        
        # If <h4>, add 'Food' or 'Exercise' notes to day's dictionary.
        elif tag.name == 'h4':
            h4_notetype = tag.get_text()
            note_text = tag.find_next_sibling('p').get_text()
            this_note_dict = {'Date': h2_date,
                              'Note Type': h4_notetype,
                              'Note': note_text
                             }
            result_dict['Notes'].append(this_note_dict)
            
        # If <h2> then it's a new day so record the date. 
        elif tag.name == 'h2':
            h2_date = tag.get_text()  

        else:
            pass
        
    return result_dict


In [5]:
def table_to_dict(html, totals_only=True):
    '''
    Purpose: Extract HTML <table> data into a dictionary.
    
    Each <table> body has 3 sections: 
    * <thead>, contains log type ('Foods' or 'Exercises').
    * <tbody>, contains multiple datasets ('Breakfast', 'Cardiovascular' etc), 
               and individual items in the dataset(s) 
               (ingredients, exercises) and item statistics ('Calories', etc).
    * <tfoot>, contains total statistics for the entire table.
    
    Args: 
        html (BeautifulSoup): HTML <table> body
        totals_only (boolean): return daily statistic totals if True, 
                               otherwise include log entry details
        
    Returns:
        dictionary: a single dated log entry
        string: table type ('Foods' or 'Exercises')
    '''
    logging.debug('table_to_dict ...')
    result_log_type = ''
    result_dict = {}  

    # Get column headers from <thead>'s <td> tags.
    #     <thead> <tr> <td>Foods</td> <td>Calories</td> <td>Carbs</td> ... </tr> </thead>
    columns = html.find('thead').find('tr').findChildren('td')
    
    # Get log type ('Foods' or 'Exercises') from the first column header.
    result_log_type = columns[0].get_text()  
    
    # Get the dataset type ('Breakfast' etc, or 'Cardiovascular' etc), 
    # along with all its items.
    if not totals_only:
        tbody = html.find('tbody')

        # Get the first dataset type from the first row. 
        #    <tbody> <tr> <td>Breakfast</td> </tr> ...
        first_row = tbody.find('tr', {'class': 'title'})
        logging.debug(f'first_row = {first_row}')
        this_dataset_type = first_row.findChild('td').get_text()

        # Then get the rest of the rows in <tbody>, which will be either a dataset item and its details ... 
        #    <tr> <td>'Oil, avocado, 1 tbsp</td><td>124</td><td>0g</td>... </tr>  
        # or the next dataset type followed by its items.
        table_rows = first_row.find_next_siblings('tr')

        this_item_dict = {}  # dictionary of an item (e.g. 'Oil, avocado')
        this_item_list = []  # list of item dictionaries 

        for row in table_rows:
            # If a row's tag (<tr>) has a class, then it's a new dataset type.
            #     Append the previous dataset's item list to the result_dict, 
            #     then reset the dataset_type to this new one.
            if row.has_attr('class'):  
                result_dict[this_dataset_type] = this_item_list 
                this_dataset_type = row.findChild('td').get_text()

            # Else this row is an item, so parse and add it to the current dataset's 
            # list of items.
            else:           
                # The first column contains the item name.
                item = row.findChild('td')
                item_text = item.get_text()

                # 'item' contains both it's name and amount ('Oil, avocado, 1 tbsp').
                this_item_dict['Item Name'] = ','.join(item_text.split(',')[:-1])
                this_item_dict['Amount'] = item_text.split(',')[-1:][0].strip()

                # The rest of the columns contain the item stats. 
                stats = item.find_next_siblings('td')
                stats_list = []
                for stat in stats:
                    stats_list.append(stat.get_text())
                this_item_dict['Stats'] = stats_list         

                this_item_list.append(this_item_dict)
                this_item_dict = {}

            # log the final dataset's items
            result_dict[this_dataset_type] = this_item_list 
        
        
    # Get summary totals from <tfoot>. Skip the first column, which is just 'TOTAL:'.
    table_totals = html.find('tfoot').find('tr').findChildren('td')
    result_dict['Totals'] = [x.get_text() for x in table_totals[1:]]
    
    return result_dict, result_log_type


In [6]:
def flatten_logs(log_list, dataset_type='Foods'):
    '''
    Purpose: Flatten a list of daily logs (dictionaries) into a dataframe.   
    
    `get_daily_logs` function extracts daily log entries into a list 
    of dictionaries (each daily log entry a dictionary). Flatten each 
    individual log entry (e.g. food item or exercise) into a dataframe row.
    
    Args: 
        log_list (list): list embedded with dictionaries
        dataset_type (str): 'Foods' or 'Exercises'
        
    Returns:
        dataframe: a row for each log entry 
    '''
    
    logging.debug('flatten_logs ...')
    # Get stats columns for this dataset_type
    stats_list = report_meta_df[dataset_type]['Stats List']
    # Initialize result dataframe 
    result_df = pd.DataFrame(columns=['Date','Entry','Item','Amount'] + stats_list)
    #r_df = result_df.copy()
    notlist = ['Date','Totals']
    for log in log_list:
        # For all columns except 'Date' and 'Totals', flatten 
        # each column value (they are dictionaries)
        for key in [x for x in log.keys() if x not in notlist]:
            for entry in log[key]:
                """
                r_df = result_df.append({'Date': log['Date'],
                                              'Entry': key,
                                              'Item': entry['Item Name'],
                                              'Amount': entry['Amount']}, 
                                             ignore_index=True)
                """
                entry_dict = {'Date': log['Date'],
                              'Entry': key,
                              'Item': entry['Item Name'],
                              'Amount': entry['Amount']}
                stats_dict = dict(zip(stats_list, entry['Stats']))
                combined_dict = dict(entry_dict, **stats_dict)
                result_df = result_df.append(combined_dict, 
                                             ignore_index=True)
                

    return result_df

# Main

In [7]:
if __name__ == '__main__':
    
    logging.debug('main ...')  
    
    # Load soup
    with open(data_file, 'r') as f:
        page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    
    # Get report meta data. Useful in `flatten_logs` function.
    report_meta_df = get_report_meta(soup)

    # Get log entry details.
    daily_log_dict = get_daily_logs(soup, totals_only=False)
    
    # Flatten 'food' and 'exercise' dictionaries. 
    logging.debug('Flatten exercise log ...')
    exercise_log_flat = flatten_logs(daily_log_dict['Exercises'], 'Exercises')
    logging.debug('Flatten food log ...')
    food_log_flat = flatten_logs(daily_log_dict['Foods'], 'Foods')
    # Notes (if any) are already flat, just convert the Notes dict to a df.
    if len(daily_log_dict['Notes']) > 0:
        logging.debug('Extract notes_log_df ...')
        notes_log_df = pd.DataFrame(daily_log_dict['Notes']) #.set_index(['Date'])

# Explore our new data structures

In [8]:
report_meta_df.head()

,Date From,Date To,Foods,Exercises
Stats List,2019-01-01,2019-01-14,"[Calories, Carbs, Fat, Protein, Cholest, Sodium, Sugars, Fiber]","[Calories, Minutes, Sets, Reps, Weight]"


In [9]:
daily_log_dict['Exercises']

[{'Cardiovascular': [{'Amount': 'walking dog',
    'Item Name': 'Walking, 3.0 mph, mod. pace',
    'Stats': ['124', '20', '\xa0']}],
  'Date': 'January 1, 2019',
  'Totals': ['124', '20', '0', '0', '0']},
 {'Date': 'January 2, 2019',
  'Strength Training': [{'Amount': 'Bent Arm Barbell Pullover',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']},
   {'Amount': 'Leg Curls',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']},
   {'Amount': 'Leg Extension',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']}],
  'Totals': ['0', '0', '9', '36', '0']},
 {'Cardiovascular': [{'Amount': 'walking dog',
    'Item Name': 'Walking, 3.0 mph, mod. pace',
    'Stats': ['124', '20', '\xa0']},
   {'Amount': 'light/moderate effort',
    'Item Name': 'Swimming laps, freestyle',
    'Stats': ['394', '30', '\xa0']}],
  'Date': 'January 3, 2019',
  'Totals': ['518', '50', '0', '0', '0']},
 {'Date': 'January 4, 2019',
  'Strength Training': [{'Amount': 'Bent Arm Barbell Pullover

In [10]:
# helper function
def print_log_dates(data, datatype):
    """Print how many days were logged, and what type of log."""
    dates = data.unique().tolist()
    print(f'\n{len(dates)} days of {datatype} logs between {dates[0]} and {dates[-1]}')
    return None
# verify count of logged days
print_log_dates(food_log_flat['Date'], 'food')
print_log_dates(exercise_log_flat['Date'], 'exercise')
print_log_dates(notes_log_df['Date'], 'note')


14 days of food logs between January 1, 2019 and January 14, 2019

9 days of exercise logs between January 1, 2019 and January 14, 2019

3 days of note logs between January 1, 2019 and January 11, 2019


# Save data to files

In [11]:
def dump_data(filename, df):
    """Write CSV file with dataframe contents. Return None."""
    df.to_csv(filename, index=False)
    return None

In [12]:
foodfile = output_dir + 'food_' + output_file
exercisefile = output_dir + 'exercise_' + output_file
notefile = output_dir + 'notes_' + output_file

dump_data(foodfile, food_log_flat)
dump_data(exercisefile, exercise_log_flat)
dump_data(notefile, notes_log_df)

In [13]:
# test output files by reloading
def load_data(filename):
    """Return dataframe loaded with pickled file."""
    return pd.read_csv(filename)

In [14]:
food_testdf = load_data(foodfile)
exercise_testdf = load_data(exercisefile)
note_testdf = load_data(notefile)

In [15]:
food_testdf.head()
exercise_testdf.head()
note_testdf.head()

,Date,Entry,Item,Amount,Calories,Carbs,Fat,Protein,Cholest,Sodium,Sugars,Fiber
0,"January 1, 2019",Breakfast,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g
1,"January 1, 2019",Breakfast,Sweet Potato Soup,2 serving(s),638,103g,15g,32g,52mg,"3,436mg",40g,17g
2,"January 1, 2019",Breakfast,Thompson Raisins - Raisin,1 c,400,100g,0g,4g,0mg,40mg,92g,8g
3,"January 1, 2019",Breakfast,"Seeds, sunflower seed kernels, dry roasted, with salt added",1 cup,745,31g,64g,25g,0mg,838mg,3g,12g
4,"January 1, 2019",Dinner,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g


,Date,Entry,Item,Amount,Calories,Minutes,Sets,Reps,Weight
0,"January 1, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN
1,"January 2, 2019",Strength Training,NaN,Bent Arm Barbell Pullover,,3,12,\n,NaN
2,"January 2, 2019",Strength Training,NaN,Leg Curls,,3,12,\n,NaN
3,"January 2, 2019",Strength Training,NaN,Leg Extension,,3,12,\n,NaN
4,"January 3, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN


,Date,Note,Note Type
0,"January 1, 2019",Starting Whole30 today.,Food Notes
1,"January 1, 2019","Also, did a lot of hauling today.",Exercise notes
2,"January 8, 2019",Symptom - Knee Pain,Exercise notes
3,"January 11, 2019",Symptom - Knee Pain,Exercise notes
